In [ ]:
import sys
sys.path.append("../..")

## Proof of Concept: Single Kernel Test Application

To test out a new kernel, the `SingleKernelApp` class would allow users to quickly test the kernel by running it on the NPU.

For example, to create a test application for the Inverse kernel, it is 1 LoC to create the application with `invapp = SingleKernelApp(Inverse)`

In [ ]:
from npu.build.appbuilder import AppBuilder
from npu.build.kernel import Kernel
import numpy as np

class SingleKernelApp(AppBuilder):
    def __init__(self, kernelfx):

        if isinstance(kernelfx, type):
            self.kernelfx = kernelfx()
        elif isinstance(kernelfx, Kernel) and callable(kernelfx):
            self.kernelfx = kernelfx
        else:
            raise Exception("kernelfx must be a Kernel or a Kernel instance")

        super().__init__()

    def callgraph(self, *args, **kwargs):
        return self.kernelfx(*args, **kwargs)

In [ ]:
iarray = np.arange(16).reshape((2, 8))
iarray

array([[ 0,  1,  2,  3,  4,  5,  6,  7],
       [ 8,  9, 10, 11, 12, 13, 14, 15]])

In [ ]:
from npu.lib.kernels import Inverse

# The App is constructed and then its behavioral model called
invapp = SingleKernelApp(Inverse)
invapp(iarray, iarray.nbytes)

array([[255, 254, 253, 252, 251, 250, 249, 248],
       [247, 246, 245, 244, 243, 242, 241, 240]])

In [ ]:
invapp.build(iarray, iarray.nbytes)

Using cached inverse kernel object file...
Building the xclbin...
Successfully Building Application... SingleKernelApp.xclbin & SingleKernelApp.seq delivered


In [ ]:
from npu.runtime import AppRunner

app = AppRunner('SingleKernelApp.xclbin') 
inbuffer = app.allocate(shape=iarray.shape, dtype=iarray.dtype)
outbuffer = app.allocate(shape=iarray.shape, dtype=iarray.dtype)

inbuffer.sync_to_npu()
app(inbuffer, outbuffer)
outbuffer.sync_from_npu()

outbuffer

In [ ]:
import numpy as np
class SingleKernelRunner:
    def __init__(self, bufs) -> None:
        self.bufs = [np.copy(b) for b in bufs]

    def run(self):   
                 
        app = AppRunner('SingleKernelApp.xclbin') 

        appbufs = [app.allocate(shape=b.shape, dtype=b.dtype) for b in self.bufs]
        _ = [b.sync_to_npu() for b in appbufs]

        app(*appbufs)

        _ = [b.sync_from_npu() for b in appbufs]
        _ = [print(b) for b in appbufs]
            

invapp = SingleKernelRunner(iarray, np.copy(iarray))
invapp.run()


In [ ]:

import numpy as np
from npu.lib.kernels import Inverse

a = np.arange(16)
invappbuilder = SingleKernelApp(Inverse)
invappbuilder.build(a, a.nbytes)

invapp = SingleKernelRunner(a, a)
invapp.run()